#  Late Night Revelers

C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


|Code      | Description                                                                                  |
|:-------- |:----------------------------------------------------------------------------------------------- | 
| C/A      | Control Area (A002)                                                                          | 
| UNIT     | Remote Unit for a station (R051)                                                             | 
| SCP      | Subunit Channel Position represents an specific address for a device (02-00-00)              | 
| STATION  | Represents the station name the device is located at                                         | 
| LINENAME | Represents all train lines that can be boarded at this station                               | 
|          |   Normally lines are represented by one character.  LINENAME 456NQR repersents train server for | 
|          |   4, 5, 6, N, Q, and R trains.                                                                  | 
| DIVISION | Represents the Line originally the station belonged to BMT, IRT, or IND                      | 
| DATE     | Represents the date (MM-DD-YY)                                                               |   
| TIME     | Represents the time (hh:mm:ss) for a scheduled audit event                                   | 
| DESc     | Represents the "REGULAR" scheduled audit event (Normally occurs every 4 hours)                | 
|          |   1. Audits may occur more than 4 hours due to planning, or troubleshooting activities.         | 
|          |   2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. |
|ENTRIES   | The cumulative entry register value for a device|
|EXIST     | The cumulative exit register value for a device|


In [32]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

# enables inline plots, without it, plots don't show up in the notebook
%matplotlib inline

In [33]:
data = pd.read_csv('../data/mta/turnstile.csv', parse_dates=['Datetime'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2533425 entries, 0 to 2533424
Data columns (total 12 columns):
C/A         object
UNIT        object
SCP         object
STATION     object
LINENAME    object
DIVISION    object
DATE        object
TIME        object
DESC        object
ENTRIES     int64
EXITS       int64
Datetime    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 231.9+ MB


In [34]:
data.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Datetime
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/12/2016,03:00:00,REGULAR,5583673,1884949,2016-03-12 03:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/12/2016,07:00:00,REGULAR,5583689,1884968,2016-03-12 07:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/12/2016,11:00:00,REGULAR,5583785,1885067,2016-03-12 11:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/12/2016,15:00:00,REGULAR,5584037,1885157,2016-03-12 15:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/12/2016,19:00:00,REGULAR,5584482,1885250,2016-03-12 19:00:00
5,A002,R051,02-00-00,59 ST,NQR456,BMT,03/12/2016,23:00:00,REGULAR,5584768,1885284,2016-03-12 23:00:00
6,A002,R051,02-00-00,59 ST,NQR456,BMT,03/13/2016,04:00:00,REGULAR,5584835,1885305,2016-03-13 04:00:00
7,A002,R051,02-00-00,59 ST,NQR456,BMT,03/13/2016,08:00:00,REGULAR,5584851,1885325,2016-03-13 08:00:00
8,A002,R051,02-00-00,59 ST,NQR456,BMT,03/13/2016,12:00:00,REGULAR,5584909,1885403,2016-03-13 12:00:00
9,A002,R051,02-00-00,59 ST,NQR456,BMT,03/13/2016,16:00:00,REGULAR,5585119,1885458,2016-03-13 16:00:00


# Extraction Deltas

The turnstiles have cumulative results, which is reset time-to-time. In order to obtain the number of passenger at the station in a specific interval, we must calculate the variance.

In [35]:
tmp = data.sort_values(['C/A', 'UNIT', 'STATION','SCP', 'Datetime'],
                       ascending =True, kind=' mergesort').copy()
tmp['Delta_ENTRIES'] = np.nan
tmp['Delta_EXITS'] = np.nan
tmp = tmp.groupby(['C/A', 'UNIT', 'STATION','SCP','Datetime']).sum()

print(tmp.shape)
tmp.sample(10)

(2533407, 4)


,,,,,ENTRIES,EXITS,Delta_ENTRIES,Delta_EXITS
C/A,UNIT,STATION,SCP,Datetime,,,,
R501,R054,5 AVE,00-00-06,2016-02-08 07:00:00,1377137,623489,NaN,NaN
A049,R088,CORTLANDT ST,02-05-01,2015-12-26 12:00:00,638058496,5177432,NaN,NaN
G001,R151,CONEY IS-STILLW,00-00-00,2016-03-24 05:00:00,2231600,4969613,NaN,NaN
R161B,R452,72 ST,00-03-03,2016-02-08 16:00:00,14459318,4762382,NaN,NaN
R119,R320,CANAL ST,00-00-00,2016-01-14 08:00:00,518129,209715,NaN,NaN
R645,R110,FLATBUSH AV-B.C,00-06-01,2016-02-24 07:56:10,1519898,418709,NaN,NaN
N338,R128,SUTPHIN BLVD,01-06-01,2016-03-11 00:00:00,1941930,1593194,NaN,NaN
PTH03,R552,JOURNAL SQUARE,00-01-04,2016-03-14 05:18:27,121478,82771,NaN,NaN
R403,R446,BROOK AV,01-00-00,2016-01-08 20:00:00,2508675,8003603,NaN,NaN


In [ ]:
# Create index to calculate Variation
key_index = tmp.reset_index().ix[:, ['C/A', 'UNIT', 'STATION','SCP'] ].drop_duplicates().copy()
key_index.sort_values(['C/A', 'UNIT', 'STATION','SCP'], ascending =True, kind=' mergesort')
key_index = [list(key_index.iloc[i,:]) for i in range(key_index.shape[0])]

key_index[:5]

[['A002', 'R051', '59 ST', '02-00-00'],
 ['A002', 'R051', '59 ST', '02-00-01'],
 ['A002', 'R051', '59 ST', '02-03-00'],
 ['A002', 'R051', '59 ST', '02-03-01'],
 ['A002', 'R051', '59 ST', '02-03-02']]

In [ ]:
# Calculate Deltas.
# warning: This may take some time.
idx = pd.IndexSlice
for  _, (i0, i1, i2, i3) in  enumerate(key_index):
    px = idx[i0, i1, i2, i3,:]
    
    delta = tmp.loc[px, [idx['ENTRIES']]].shift(-1) - tmp.loc[px, [idx['ENTRIES']]]
    tmp.loc[px, [idx['Delta_ENTRIES']]] = delta['ENTRIES'].values
    
    delta = tmp.loc[px, [idx['EXITS']]].shift(-1) - tmp.loc[px, [idx['EXITS']]]
    tmp.loc[px, [idx['Delta_EXITS']]] = delta['EXITS'].values


In [ ]:
tmp.sample(10)

# Identify Anomalies

Due to machine failure and possible error during data storage, some results are negative or too high.
To ensure the "delta values" are valid, we assign unlike values as ```NaN``` values.

In [ ]:
def plot_hist(x,name):
    
    plt.figure(1,figsize=(28,10))
    mask = x > 0 & (~np.isnan(x))
    plt.subplot(131)
    plt.hist(x[mask], bins= 200)
    plt.title("Histogram of Delta " + name);
    plt.xlabel('Number of Passengers')
    plt.ylabel('Frequency')

    mask = x >= 0 & (~np.isnan(x))
    plt.subplot(132)
    plt.hist(np.log(x[mask] + 1), bins= 200)
    plt.title("Histogram of Log Delta " + name);
    plt.xlabel('Number of Passengers')
    
    mask = x > 0 & (~np.isnan(x))
    plt.subplot(133)
    plt.hist(np.log(x[mask] + 1), bins= 200)
    plt.title("Histogram of Log Delta " + name + " (no zeros)");
    plt.xlabel('Number of Passengers')
    
    print('Max: %d' % np.nanmax(x))
    print('Min: %d' % np.nanmin(x))
    

In [ ]:
plot_hist(x=tmp.Delta_ENTRIES.dropna().copy().values,name='Entries')

In [ ]:
plot_hist(x=tmp.Delta_EXITS.dropna().copy().values, name='Exits')

It clear that the distribution is very skewed, even after the log transformation.
Also, there are a high number of zeros. It may sound odd, but it is reasonable because many stations stay closed or have very low use at specific time frames.

In order to avoid removing too many values, we remove the anomalies at the turnstile then aggregate the results by station.

In [ ]:
def remove_anomalies(X):
    """This function removes unusual values"""
    
    #Set negative values to NaN
    mask = X < 0
    X[mask] = np.nan
    
    mask = X <= 0 | np.isnan(X)
    if len(X[mask]) == 0:
        return(X)
    
    #Use log to have it closer to a normal distribution
    X_log = np.log(X.copy() + 1)
    
    x_bar = np.nanmean(X_log)
    sigma = np.nanstd(X_log)
    
    # Set extreme values as NaN
    mask = X_log > (x_bar + 3*sigma)
    
    X[mask] = np.nan
    
    return(X)

In [ ]:
# Remove anomalies at turnstile level.
tmp.Delta_ENTRIES  = remove_anomalies(tmp.Delta_ENTRIES.values)
tmp.Delta_EXITS  = remove_anomalies(tmp.Delta_EXITS.values)

# idx = pd.IndexSlice
# for  _, (i0, i1, i2, i3) in  enumerate(key_index):
    
#     px = idx[i0, i1, i2, i3,:]
#     #print(tmp.Delta_ENTRIES.loc[px].sample(10))
    
#     tmp.loc[px, [idx['Delta_ENTRIES']]]  = remove_anomalies(tmp.Delta_ENTRIES.loc[px].values)
#     tmp.loc[px, [idx['Delta_EXITS']]]  = remove_anomalies(tmp.Delta_EXITS.loc[px].values)

In [ ]:
plot_hist(x=tmp.Delta_ENTRIES.dropna().copy().values,name='Entries')

In [ ]:
plot_hist(x=tmp.Delta_EXITS.dropna().copy().values,name='Exits')

# Aggregate Delta by Station

After calculating Deltas and removing anomalies, it is time to aggregate results by station.
First, we round the ```datetime``` to the nearest hour.

In [ ]:
def round_datetime(x):
    second = x.second
    minute = x.minute
    hour = x.hour
    date= x.date()
    
    #round second
    if second < 30:
        second = 0
    else:
        second = 0
        minute += 1
        
    #round minute   
    if minute < 30:
        minute = 0
    else:
        minute = 0
        hour += 1
    
    #adjust day shift    
    if hour > 23:
        hour = 0
        date += dt.timedelta(days=1)
        
    return(dt.datetime(date.year, date.month, date.day, hour, minute, second))

In [ ]:
# Round time interval to the nearest hour
tmp.reset_index(inplace=True)

print('Total number of intervals before adjust: %12d' % tmp.Datetime.nunique())
tmp.Datetime = tmp.Datetime.apply(round_datetime)
print('Total number of intervals after adjust:  %12d' % tmp.Datetime.nunique())

In [ ]:
tmp.drop(['ENTRIES', 'EXITS'], axis=1, inplace=True) 
tmp.sample(10)

In [ ]:
total = tmp.shape[0]
mask_exits  = np.isnan(tmp.Delta_EXITS.values)
print('%4.2f%% of the data have Delta Exits missing' % 
      (np.sum(mask_exits) / total *100))

mask_entries  = np.isnan(tmp.Delta_ENTRIES.values)
print('%4.2f%% of the data have Delta Entries missing' % 
      (np.sum(mask_entries) / total *100))

mask_all = mask_exits | mask_entries
print('%4.2f%% of the data have Delta Entries OR Delta Exits  missing' % 
      (np.sum(mask_all) / total *100))

In [ ]:
# Remove missing deltas
tmp = tmp.ix[~mask_all,:]
tmp.info()

In [ ]:
tmp['Delta_total'] = tmp.loc[:,'Delta_ENTRIES'] + tmp.loc[:,'Delta_EXITS'] 

In [ ]:
tmp = tmp.groupby(['STATION', 'Datetime'], as_index=False).sum()
tmp = tmp.sort_values(['STATION', 'Datetime'], ascending=[True, True])

In [ ]:
tmp.sample(10)

# Select Specific Time and Day

In [23]:
INTERVAL = [dt.datetime(year=2000,month=1, day=1, hour=12, minute=0, second=0),
           dt.datetime(year=2000,month=1, day=1, hour=5, minute=0, second=0)]

def is_late_night(x):
    if x >= INTERVAL[0] or x <= INTERVAL[1]:
        return(True)
    else:
        return(False)

DAY_OF_WEEK = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday', 'Monday']
def get_day_of_week(x):
    if x.hour >= 22:
        return DAY_OF_WEEK[x.weekday() + 1]
    else:
        return DAY_OF_WEEK[x.weekday()]
    
def is_weekend(x):
    if x.hour >= 22:
        if (x.weekday() + 1 < 5) or (x.weekday() + 1) == 7:
            return(False)
        else:
            return(True)
    else:
        if x.weekday() < 5:
            return(False)
        else:
            return(True)


#data['DAYOFWEEK'] = data.DATETIME.apply(whatdayofweek)
tmp['Day_of_week'] = tmp.Datetime.apply(get_day_of_week)
tmp['Weekend'] = tmp.Datetime.apply(is_weekend)

In [24]:
mask = tmp.Datetime.apply(is_late_night)
tmp = tmp.ix[mask, :]
tmp.sample(10)

,STATION,Datetime,Delta_ENTRIES,Delta_EXITS,Delta_total,Day_of_week,Weekend
77731,ASTOR PL,2016-03-18 05:00:00,2369.0,2447.0,4816.0,Friday,False
114997,CITY / BUS,2016-01-13 08:00:00,3431.0,461.0,3892.0,Wednesday,False
230811,THIRTY THIRD ST,2016-01-08 06:00:00,451.0,376.0,827.0,Friday,False
35100,28 ST,2016-01-03 20:00:00,2242.0,1362.0,3604.0,Sunday,True
164263,JFK JAMAICA CT1,2016-02-10 12:00:00,1838.0,1494.0,3332.0,Wednesday,False
181642,MONTROSE AV,2016-01-05 23:00:00,189.0,280.0,469.0,Wednesday,False
232120,THIRTY THIRD ST,2016-03-14 19:00:00,1298.0,220.0,1518.0,Monday,False
118770,CLINTON-WASH AV,2016-01-23 20:00:00,130.0,27.0,157.0,Saturday,True
38367,33 ST,2016-01-04 08:00:00,5932.0,9726.0,15658.0,Monday,False
190257,NEW LOTS AV,2016-03-15 20:00:00,541.0,913.0,1454.0,Tuesday,False


In [31]:
tmp = tmp.groupby(['STATION', 'Day_of_week'], as_index=False).mean().head()

# #Busiest Stations on Weekend
mask = tmp.Weekend.values
tmp.sort_values(['Delta_total'], ascending=False
               ).drop_duplicates('STATION').ix[:,['STATION','Delta_total']].head(20)

,STATION,Delta_total
2,103 ST,3286318.0
0,1 AV,2988931.0
4,103 ST-CORONA,2202566.0
